
<center>
    <img src="logo.png" height="300" width="500"/>
        <h1 style='color:#142b5a; font-weight: bold; font-family: Calibri;font-size: 40px; '>  Prediction of together-bought products with Apriori  </h1>
</center>


 <p style='color:#142b5a; font-weight: bold; font-family: Calibri; font-size:25px;'>🎯 ** Objectives **</p>
 <br>
<div style='color:#142b5a; font-weight: light; font-family: Calibri;'>In this Notebook we will predict the profit with diffrent methods.</div>

<p style='color:#142b5a; font-weight: bold; font-family: Calibri; font-size:25px;'>✔️ ** Outline **</p> 
<br>
<a href="#import" style='color:#142b5a; font-weight: light; font-family: Calibri;'> • Importing necessary libraries</a>
<br>
<br>
<a href="#clean" style='color:#142b5a; font-weight: light; font-family: Calibri;'> • Importing the dataset</a>
<br>
<br>
<a href="#discover" style='color:#142b5a; font-weight: light; font-family: Calibri;'> • Exploring the dataset</a>
<br>
<br>
<a href="#rename" style='color:#142b5a; font-weight: light; font-family: Calibri;'> • Renaming columns </a>
<br>
<a href="#cuf" style='color:#142b5a; font-weight: light; font-family: Calibri;'>• Chosing usefull features</a>
<br> 
<a href="#splitting" style='color:#142b5a; font-weight: light; font-family: Calibri; padding:0 0 0 20px'>- Spliting for a better using</a>
<br>
<a href="#listprod" style='color:#142b5a; font-weight: light; font-family: Calibri; padding:0 0 0 20px'>- List of bought-together products</a>
<br>
<a href="#Apriori" style='color:#142b5a; font-weight: light; font-family: Calibri; padding:0 0 0 20px'>- Application of Apriori</a>
<br>
<a href="#Result" style='color:#142b5a; font-weight: light; font-family: Calibri;padding:0 0 0 20px'> - Apriori Results</a>
        
  
</div>
<br>


<div style='font-size:100%;'>
    <a id='import'></a>
    <h1 style='color: black; font-weight: bold; font-family: Calibri;'>
        <center> Importing necessary libraries 📚 </center>
    </h1>
 
</div>

In [ ]:
#Installer apyori

!pip install apyori

In [3]:
#Importer les Libraries
import numpy as np
import pyodbc as driver
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import pyodbc
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
     
import sklearn             
import os
import matplotlib
from tqdm.notebook import tqdm
import pygrametl
from pygrametl.tables import Dimension
from pygrametl.tables import FactTable
from apyori import apriori

%matplotlib inline 


<div style='font-size:100%;'>
    <a id='clean'></a>
    <h1 style='color: black; font-weight: bold; font-family: Calibri;'>
        <center> Importing the dataset ⬇️ </center>
    </h1>
</div>

In [58]:
import pyodbc 
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-N7M2UPR;'
                      'Database=DataProjet;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()



import pyodbc 
conn2 = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-N7M2UPR;'
                      'Database=DW_projet;'
                      'Trusted_Connection=yes;')

cursor2 = pygrametl.ConnectionWrapper(connection=conn2)

# Suppress all warnings
import warnings
warnings.filterwarnings("ignore")

#Reading SQL query into a DataFrame 

#Fact Sales
SQL_Query = pd.read_sql_query('''select * FROM DW_projet.dbo.FactSales''', conn2)

data = pd.DataFrame(SQL_Query)

<div style='font-size:100%;'>
    <a id='discover'></a>
    <h1 style='color: black; font-weight: bold; font-family: Calibri;'>
        <center> Exploring the dataset 🧐 </center>
    </h1>
</div>

In [59]:
data.head()

,FK_Customer,FK_Product,FK_Ship,FK_Date_Order,FK_Date_Ship,FK_Order,FK_Localisation,Quantity,Profit,Sales,Discount,Returned,Ship_Coast,Ship_Duration
0,52246,20958,17,20120703,20120706,60952,13621,7,-4.210500,44.929501,0.45,NO,6.91,3
1,52246,20993,20,20150107,20150111,61999,11734,5,-10.440000,23.760000,0.45,NO,2.84,4
2,52246,21764,17,20120703,20120706,60952,13621,2,-23.528999,51.710999,0.45,NO,1.77,3
3,52246,21796,17,20120703,20120706,60952,13621,13,-141.960007,278.850006,0.45,NO,30.01,3
4,52246,22091,17,20120703,20120706,60952,13621,1,-8.532000,29.238001,0.45,NO,6.66,3


In [60]:
SQL_Query = pd.read_sql_query('''select Product_PK,[Product Name] FROM DW_Projet.dbo.DimProduct''', conn2)
dimProduct = pd.DataFrame(SQL_Query)

#=== <div style='font-size:100%;'>
    <a id='rename'></a>
    <h1 style='color: black; font-weight: bold; font-family: Calibri;'>
        <center> Renaming columns ⬇️ </center>
    </h1>
</div>

In [61]:
dimProduct=dimProduct.rename(columns={'Product_PK':'FK_Product'} )
data['FK_Product']=pd.merge(data,dimProduct,on='FK_Product',how='left')['Product Name']

In [62]:
data

,FK_Customer,FK_Product,FK_Ship,FK_Date_Order,FK_Date_Ship,FK_Order,FK_Localisation,Quantity,Profit,Sales,Discount,Returned,Ship_Coast,Ship_Duration
0,52246,"Avery Shipping Labels, Alphabetical",17,20120703,20120706,60952,13621,7,-4.210500,44.929501,0.450,NO,6.91,3
1,52246,"Hon File Folder Labels, Laser Printer Compatible",20,20150107,20150111,61999,11734,5,-10.440000,23.760000,0.450,NO,2.84,4
2,52246,"Elite Shears, High Speed",17,20120703,20120706,60952,13621,2,-23.528999,51.710999,0.450,NO,1.77,3
3,52246,"Fiskars Trimmer, Serrated",17,20120703,20120706,60952,13621,13,-141.960007,278.850006,0.450,NO,30.01,3
4,52246,"SanDisk Numeric Keypad, USB",17,20120703,20120706,60952,13621,1,-8.532000,29.238001,0.450,NO,6.66,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51246,69658,"Stockwell Rubber Bands, Metal",19,20120919,20120921,63264,12004,4,11.940000,59.939999,0.100,NO,2.31,2
51247,69659,"Tenex Frame, Black",20,20131126,20131203,70054,14903,2,7.304000,87.984001,0.400,NO,9.11,7
51248,69659,"Sharp Fax Machine, Laser",19,20150618,20150621,72408,13553,3,69.737396,590.117371,0.002,NO,106.68,3
51249,69660,"SanDisk Parchment Paper, Premium",20,20131212,20131216,67182,12867,8,25.200001,133.679993,0.000,NO,4.76,4


<div style='font-size:100%;'>
    <a id='cuf'></a>
    <h1 style='color: black; font-weight: bold; font-family: Calibri;'>
        <center> Chosing usefull features  ⬇️ </center>
    </h1>
</div>

In [63]:

orders=data[['FK_Order','FK_Product']]
orders

,FK_Order,FK_Product
0,60952,"Avery Shipping Labels, Alphabetical"
1,61999,"Hon File Folder Labels, Laser Printer Compatible"
2,60952,"Elite Shears, High Speed"
3,60952,"Fiskars Trimmer, Serrated"
4,60952,"SanDisk Numeric Keypad, USB"
...,...,...
51246,63264,"Stockwell Rubber Bands, Metal"
51247,70054,"Tenex Frame, Black"
51248,72408,"Sharp Fax Machine, Laser"
51249,67182,"SanDisk Parchment Paper, Premium"


In [64]:
data1=data.copy()

<div style='font-size:100%;'>
    <a id='splitting'></a>
    <h1 style='color: black; font-weight: bold; font-family: Calibri;'>
        <center> Spliting for a better using ⬇️ </center>
    </h1>
</div>

In [65]:
data1['FK_Product'] = data['FK_Product'].str.split(',').str[0]

In [66]:
data1

,FK_Customer,FK_Product,FK_Ship,FK_Date_Order,FK_Date_Ship,FK_Order,FK_Localisation,Quantity,Profit,Sales,Discount,Returned,Ship_Coast,Ship_Duration
0,52246,Avery Shipping Labels,17,20120703,20120706,60952,13621,7,-4.210500,44.929501,0.450,NO,6.91,3
1,52246,Hon File Folder Labels,20,20150107,20150111,61999,11734,5,-10.440000,23.760000,0.450,NO,2.84,4
2,52246,Elite Shears,17,20120703,20120706,60952,13621,2,-23.528999,51.710999,0.450,NO,1.77,3
3,52246,Fiskars Trimmer,17,20120703,20120706,60952,13621,13,-141.960007,278.850006,0.450,NO,30.01,3
4,52246,SanDisk Numeric Keypad,17,20120703,20120706,60952,13621,1,-8.532000,29.238001,0.450,NO,6.66,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51246,69658,Stockwell Rubber Bands,19,20120919,20120921,63264,12004,4,11.940000,59.939999,0.100,NO,2.31,2
51247,69659,Tenex Frame,20,20131126,20131203,70054,14903,2,7.304000,87.984001,0.400,NO,9.11,7
51248,69659,Sharp Fax Machine,19,20150618,20150621,72408,13553,3,69.737396,590.117371,0.002,NO,106.68,3
51249,69660,SanDisk Parchment Paper,20,20131212,20131216,67182,12867,8,25.200001,133.679993,0.000,NO,4.76,4


In [67]:
orders=data1[['FK_Order','FK_Product']]  #group by
orders

,FK_Order,FK_Product
0,60952,Avery Shipping Labels
1,61999,Hon File Folder Labels
2,60952,Elite Shears
3,60952,Fiskars Trimmer
4,60952,SanDisk Numeric Keypad
...,...,...
51246,63264,Stockwell Rubber Bands
51247,70054,Tenex Frame
51248,72408,Sharp Fax Machine
51249,67182,SanDisk Parchment Paper


<div style='font-size:100%;'>
    <a id='listprod'></a>
    <h1 style='color: black; font-weight: bold; font-family: Calibri;'>
        <center> List of bought-together products⬇️ </center>
    </h1>
</div>

In [68]:
dict={}
for j in  orders.index:
    if(not orders['FK_Order'][j] in dict):
         dict.update( [(orders['FK_Order'][j], [orders['FK_Product'][j]])] )
    else:
        dict[orders['FK_Order'][j]].append(orders['FK_Product'][j])

           
dict

{60952: ['Avery Shipping Labels',
  'Elite Shears',
  'Fiskars Trimmer',
  'SanDisk Numeric Keypad'],
 61999: ['Hon File Folder Labels', 'Konica Calculator'],
 57372: ['Sauder Library with Doors'],
 59368: ['Bush Floating Shelf Set',
  'Binney & Smith Pencil Sharpener',
  'Cardinal Binder Covers',
  'Smead Lockers',
  'Tenex Folders',
  'Apple Speaker Phone',
  'Nokia Speaker Phone',
  'Samsung Audio Dock'],
 67445: ['Advantus Stacking Tray'],
 58297: ['Boston Canvas'],
 58296: ['Avery Index Tab', 'Eldon Box', 'Motorola Headset'],
 54729: ['Belkin 325VA UPS Surge Protector',
  'Avery Binding System Hidden Tab Executive Style Index Sets'],
 53580: ["Acco Banker's Clasps",
  'High Speed Automatic Electric Letter Opener',
  'Verbatim 25 GB 6x Blu-ray Single Layer Recordable Disc',
  'Polycom VVX 310 VoIP phone'],
 51800: ['Fellowes Advanced Computer Series Surge Protectors',
  'GBC DocuBind 200 Manual Binding Machine'],
 52641: ['Tenex Personal Project File with Scoop Front Design'],
 518

In [69]:
listP=[]
for j in  dict.values():
    listP.append(j)
listP

[['Avery Shipping Labels',
  'Elite Shears',
  'Fiskars Trimmer',
  'SanDisk Numeric Keypad'],
 ['Hon File Folder Labels', 'Konica Calculator'],
 ['Sauder Library with Doors'],
 ['Bush Floating Shelf Set',
  'Binney & Smith Pencil Sharpener',
  'Cardinal Binder Covers',
  'Smead Lockers',
  'Tenex Folders',
  'Apple Speaker Phone',
  'Nokia Speaker Phone',
  'Samsung Audio Dock'],
 ['Advantus Stacking Tray'],
 ['Boston Canvas'],
 ['Avery Index Tab', 'Eldon Box', 'Motorola Headset'],
 ['Belkin 325VA UPS Surge Protector',
  'Avery Binding System Hidden Tab Executive Style Index Sets'],
 ["Acco Banker's Clasps",
  'High Speed Automatic Electric Letter Opener',
  'Verbatim 25 GB 6x Blu-ray Single Layer Recordable Disc',
  'Polycom VVX 310 VoIP phone'],
 ['Fellowes Advanced Computer Series Surge Protectors',
  'GBC DocuBind 200 Manual Binding Machine'],
 ['Tenex Personal Project File with Scoop Front Design'],
 ['Master Caster Door Stop', 'Staples'],
 ['Motorola Signal Booster'],
 ['BIC Hig

In [70]:
type(listP)

list

<div style='font-size:100%;'>
    <a id='Apriori'></a>
    <h1 style='color: black; font-weight: bold; font-family: Calibri;'>
        <center> Application of Apriori ⬇️ </center>
    </h1>
</div>

In [71]:
#Appliquer la méthode Apriori

In [72]:
association_rules = apriori(listP, min_support=0.00008, min_confidence=0.1, min_lift=1, min_length=2)
association_results = list(association_rules)

In [73]:
#Afficher les résultats de la méthode Apriori

In [74]:
#Afficher le nombre de règles 

In [75]:
print(len(association_results))

10


In [76]:
#Afficher la première règle
print(association_results[1])

RelationRecord(items=frozenset({'Hamilton Beach Blender', 'Acco Index Tab'}), support=0.0001554726368159204, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Hamilton Beach Blender'}), items_add=frozenset({'Acco Index Tab'}), confidence=0.125, lift=21.583892617449667)])


<div style='font-size:100%;'>
    <a id='Result'></a>
    <h1 style='color: black; font-weight: bold; font-family: Calibri;'>
        <center> Printing Apriori Results⬇️ </center>
    </h1>
</div>

In [77]:
# Ce script sert à afficher la règle, le support, la confiance et le lift d'une manière claire

In [78]:
for item in association_results:

    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    print("Rule: " + str(items[0]) + " -> " + items[1])

    #second index of the inner list
    print("Support: " + str(item[1]))

    #third index of the list located at 0th
    #of the third index of the inner list

    print("Confidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("=====================================")
   

Rule: Staples -> #10 White Business Envelopes
Support: 0.0001166044776119403
Confidence: 0.42857142857142855
Lift: 49.66795366795366
Rule: Hamilton Beach Blender -> Acco Index Tab
Support: 0.0001554726368159204
Confidence: 0.125
Lift: 21.583892617449667
Rule: Staples -> Adjustable Depth Letter/Legal Cart
Support: 0.0001166044776119403
Confidence: 0.2727272727272727
Lift: 31.606879606879602
Rule: StarTech Receipt Printer -> Eldon Box
Support: 0.0001166044776119403
Confidence: 0.1
Lift: 18.643478260869568
Rule: Hon Olson Stacker Chairs -> Staples
Support: 0.0001166044776119403
Confidence: 0.75
Lift: 86.91891891891892
Rule: Staples -> Hoover Shoulder Vac Commercial Portable Vacuum
Support: 0.0001166044776119403
Confidence: 0.5
Lift: 57.945945945945944
Rule: Staples -> KI Adjustable-Height Table
Support: 0.0001166044776119403
Confidence: 0.16666666666666669
Lift: 19.315315315315317
Rule: Staples -> Newell 34
Support: 0.0001166044776119403
Confidence: 0.42857142857142855
Lift: 49.6679536679